# Skrypt do pobierania danych aktualnych oraz historycznych + przetwarzanie i korekta.

Korekta i przetwarzanie danych wynikały z mojej analizy w notebooku data_analysis. W dokumentacji jest 60 kolumn, które raz są opisane a raz nie. Analizując trendy, dane i średnie mogłem wywnioskować która kolumna została źle wczytana, lub trzeba zastosować korekte wynikającą z danych w innych kolumnach.

Dane pobierane są w loopie jako zipy i concatowane do jednego dataframe. Można wybrać dowolny zakres lat i dowolną stację. ID stacji dla aktualnych danych z API ma inny format niż z danych historycznych.

In [1]:
import pandas as pd
import requests
import warnings
import os
import zipfile
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
STACJA_ID = "12600"  # Bielsko-Biała
BASE_URL = "https://danepubliczne.imgw.pl/api/data/synop"

In [3]:
def pobierz_aktualne_dane():
    """Pobiera aktualne dane dla stacji Bielsko-Biała"""
    url = f"{BASE_URL}/id/{STACJA_ID}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def pobierz_wszystkie_stacje():
    """Pobiera dane ze wszystkich stacji"""
    response = requests.get(BASE_URL)
    if response.status_code == 200:
        return response.json()
    return None

In [4]:
aktualne_dane = pobierz_aktualne_dane()
print("Aktualne dane dla Bielska-Białej:")
print(pd.DataFrame([aktualne_dane]))

Aktualne dane dla Bielska-Białej:
  id_stacji         stacja data_pomiaru godzina_pomiaru temperatura  \
0     12600  Bielsko Biała   2025-01-04              20        -2.6   

  predkosc_wiatru kierunek_wiatru wilgotnosc_wzgledna suma_opadu cisnienie  
0               4             180                65.7        0.1    1021.7  


In [26]:
def pobierz_dane_historyczne(rok, stacja_id='600'):
    """
    Pobiera dane historyczne dla wybranej stacji za dany rok
    """
    os.makedirs('dane', exist_ok=True)
    filename = f"{rok}_{stacja_id}_s.zip"
    url = f"https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/terminowe/synop/{rok}/{filename}"
    
    try:
        print(f"Pobieranie danych z: {url}")
        response = requests.get(url)
        
        if response.status_code == 200:
            zip_path = os.path.join('dane', filename)
            
            with open(zip_path, 'wb') as f:
                f.write(response.content)
            print(f"Zapisano plik ZIP: {zip_path}")
            
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                print("Pliki w archiwum:")
                for name in zip_ref.namelist():
                    print(f"- {name}")
                
                csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv')]
                if csv_files:
                    with zip_ref.open(csv_files[0]) as f:
                        df = pd.read_csv(
                            f,
                            encoding='cp1250',
                            sep=',',
                            quotechar='"',
                            na_values=['', '""'],
                            header=None
                        )
                        
                        print("\nPierwszy wiersz surowych danych:")
                        for i, value in enumerate(df.iloc[0]):
                            print(f"Kolumna {i}: {value}")
                            
                        print("\nSprawdzanie konkretnych kolumn:")
                        print("Wilgotność względna (kolumna 38):", df[37].head())
                        print("Suma opadów (kolumna 47):", df[46].head())
                        
                        df_clean = pd.DataFrame({
                            'data_pomiaru': pd.to_datetime(df[[2, 3, 4]].astype(str).agg('-'.join, axis=1)),
                            'godzina_pomiaru': df[5].astype(str).str.zfill(2),
                            'temperatura': pd.to_numeric(df[29], errors='coerce'),  
                            'predkosc_wiatru': pd.to_numeric(df[25], errors='coerce'),
                            'kierunek_wiatru': pd.to_numeric(df[23], errors='coerce'),
                            'wilgotnosc_wzgledna': pd.to_numeric(df[37], errors='coerce'),
                            'suma_opadu': pd.to_numeric(df[46], errors='coerce'),  # Odczytujemy wartość opadu
                            'status_opadu': df[47],  # Odczytujemy status opadu
                            'cisnienie': pd.to_numeric(df[41], errors='coerce')
                        })
                        # Przetwarzanie opadów
                        df_clean['suma_opadu'] = df_clean.apply(
                            lambda row: 0 if row['status_opadu'] == '9'  # Brak zjawiska
                            else np.nan if row['status_opadu'] == '8'  # Brak pomiaru
                            else np.nan if row['suma_opadu'] < 0  # Ujemne wartości traktujemy jako błąd
                            else row['suma_opadu'],
                            axis=1
                        )

                        df_clean['suma_opadu'] = df_clean['suma_opadu'].fillna(0)

                        df_clean = df_clean.drop('status_opadu', axis=1)

                        print("\nPo przetworzeniu - pierwsze wiersze:")
                        print(df_clean.head())
                        print("\nBrakujące wartości po przetworzeniu:")
                        print(df_clean.isnull().sum())
                        
                        return df_clean
                else:
                    print("Nie znaleziono pliku CSV w archiwum")
                    return None
            
    except Exception as e:
        print(f"Błąd podczas pobierania danych: {e}")
        return None

def pobierz_dane_wieloletnie(lata, stacja_id='600'):
    """
    Pobiera dane z wielu lat i łączy je w jeden DataFrame
    
    Args:
        lata (list): Lista lat do pobrania
        stacja_id (str): ID stacji (np. '600' dla Bielska-Białej)
    """
    wszystkie_dane = []
    
    for rok in lata:
        print(f"\nPobieranie danych za rok {rok}")
        df = pobierz_dane_historyczne(rok, stacja_id)
        if df is not None:
            wszystkie_dane.append(df)
    
    if wszystkie_dane:
        df_final = pd.concat(wszystkie_dane, ignore_index=True)
        print(f"\nŁączna liczba rekordów: {len(df_final)}")
        
        output_path = os.path.join('dane', f'dane_stacja_{stacja_id}.csv')
        df_final.to_csv(output_path, index=False)
        print(f"Zapisano połączone dane do: {output_path}")
        
        return df_final
    return None

In [27]:
lata_do_pobrania = [2020, 2021, 2022, 2023] # niestety w roku 2024 format plików się zmienił
nr_stacji = 600 # Bielsko
dane = pobierz_dane_wieloletnie(lata_do_pobrania, nr_stacji)

if dane is not None:
    print("\nPrzykładowe dane:")
    print(dane.head())
    print("\nKolumny w danych:")
    print(dane.columns.tolist())


Pobieranie danych za rok 2020
Pobieranie danych z: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/terminowe/synop/2020/2020_600_s.zip
Zapisano plik ZIP: dane/2020_600_s.zip
Pliki w archiwum:
- s_t_600_2020.csv

Pierwszy wiersz surowych danych:
Kolumna 0: 349190600
Kolumna 1: BIELSKO-BIAŁA
Kolumna 2: 2020
Kolumna 3: 1
Kolumna 4: 1
Kolumna 5: 0
Kolumna 6: 4
Kolumna 7: nan
Kolumna 8: 500
Kolumna 9: nan
Kolumna 10: 800
Kolumna 11: nan
Kolumna 12: 500/800
Kolumna 13: nan
Kolumna 14: nan
Kolumna 15: 7
Kolumna 16: nan
Kolumna 17: 10000
Kolumna 18: nan
Kolumna 19: 10475
Kolumna 20: nan
Kolumna 21: 7
Kolumna 22: nan
Kolumna 23: 251
Kolumna 24: nan
Kolumna 25: 4
Kolumna 26: nan
Kolumna 27: 0
Kolumna 28: 9.0
Kolumna 29: 2.8
Kolumna 30: nan
Kolumna 31: 0.0
Kolumna 32: 8
Kolumna 33: U
Kolumna 34: nan
Kolumna 35: 5.9
Kolumna 36: nan
Kolumna 37: 79
Kolumna 38: nan
Kolumna 39: -0.5
Kolumna 40: nan
Kolumna 41: 982.8
Kolumna 42: nan
Kolumna 43: 1032.7
Kolumna 44: na